# This notebook is for the drug/prescription information.

In [19]:
import pandas as pd
import numpy as np
import time

In [20]:
drug_file = pd.read_csv('DRUG20Q3.txt', delimiter='$')

C:\Users\G-Sprouts\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [21]:
# quantify total missing values 


null_df = pd.DataFrame(drug_file.isnull().sum().sort_values(ascending=False)).reset_index()
null_df.columns = [['column', 'missing_values']]

null_df

,column,missing_values
0,exp_dt,1889333
1,cum_dose_unit,1847041
2,cum_dose_chr,1845945
3,rechal,1600785
4,lot_num,1493003
5,dose_freq,1465144
6,nda_num,1262789
7,dose_unit,1158789
8,dose_amt,1158242
9,dose_form,1130042


In [22]:
# identify data types 

drug_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1895153 entries, 0 to 1895152
Data columns (total 20 columns):
 #   Column         Dtype  
---  ------         -----  
 0   primaryid      int64  
 1   caseid         int64  
 2   drug_seq       int64  
 3   role_cod       object 
 4   drugname       object 
 5   prod_ai        object 
 6   val_vbm        int64  
 7   route          object 
 8   dose_vbm       object 
 9   cum_dose_chr   float64
 10  cum_dose_unit  object 
 11  dechal         object 
 12  rechal         object 
 13  lot_num        object 
 14  exp_dt         object 
 15  nda_num        float64
 16  dose_amt       object 
 17  dose_unit      object 
 18  dose_form      object 
 19  dose_freq      object 
dtypes: float64(2), int64(4), object(14)
memory usage: 289.2+ MB


# Handling Missing Values

In [23]:
# Convert all values to string, then match indicies for all nan and non-nan values 
# into seperate series'



drug_file.prod_ai = drug_file.prod_ai.astype(str)

indicies = drug_file[drug_file.prod_ai != 'nan'].index
indicies2 = drug_file[drug_file.prod_ai == 'nan'].index

present = drug_file.prod_ai.loc[indicies]
needs_work = drug_file.prod_ai.loc[indicies2]



In [24]:
# Create DataFrame of 'nan' values, and attach teh series of matching indicies for which the drugname is present.
# Needs work...



needs_work_df = pd.DataFrame(needs_work)
needs_work_df['drugname'] = drug_file.drugname.loc[indicies2]

needs_work_df.drugname.value_counts()


AERIUS                                                        175
REACTINE                                                      148
TN UNSPECIFIED                                                 88
LMX                                                            86
NOVALGIN                                                       86
                                                             ... 
TOPIQUE JAUNE                                                   1
Vivelle? Dot Dis                                                1
HYCODAN [HOMATROPINE METHYLBROMIDE;HYDROCODONE BITARTRATE]      1
Oxycodone 2.5?5mg                                               1
ALUMINUM?MAGNESIUM HYDROXIDE                                    1
Name: drugname, Length: 23775, dtype: int64

# Custom Table Creation

In [161]:
new_drug_df = drug_file[['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'prod_ai', 'val_vbm', 'route', 'dose_vbm', 'cum_dose_chr']]

drugs = pd.DataFrame()

In [ ]:
# for loop for determining the amount of 'PS' (Primary suspect), 'SS' (Secondary Suspects), 'C', (concamitants), 
# or 'I' (Interacting) medications present for every unique CaseID ***6.5hr runtime***

start_time = time.time()
p = 0
s = 0
c = 0
i = 0


for x,j in enumerate(new_drug_df.caseid.unique()):
    df = new_drug_df[new_drug_df.caseid == j]


    for y in range(len(df)):
        if df.iloc[y,3] == 'PS':
            p += 1
        elif df.iloc[y,3] == 'SS':
            s += 1
        elif df.iloc[y,3] == 'c':
            c += 1
        else:
            i += 1

    drugs.loc[x, 'CaseID'] = j
    drugs.loc[x, 'Primary'] = p
    drugs.loc[x, 'Secondary'] = s
    drugs.loc[x, 'Concamitant'] = c
    drugs.loc[x, 'Interacting'] = i
        
    p,s,c,i = 0,0,0,0
end_time = time.time()

total_time = (end_time - start_time) / 60
total_time / 60

In [ ]:
# change data type of CaseID from float to integer

drugs.CaseID = drugs.CaseID.astype(int)

# export dataframe to .csv file for manipulation later

drugs.to_csv('drugs.csv', index=False)

# Mapping drugnames into numerical classes

In [166]:
class_df = pd.DataFrame(columns=['drugname', 'class_id', 'class', 'indication'])


In [167]:
# Convert all values in 'prod_ai' column to string values, and then separate non-nan values out for class mapping



drug_file.prod_ai = drug_file.prod_ai.astype(str)
drug_file.prod_ai = drug_file.prod_ai.map(lambda x: x.replace('.', ''))
indicies = drug_file[drug_file.prod_ai != 'nan'].index
present = drug_file.prod_ai.loc[indicies]
class_df.drugname = present

In [168]:
len(drug_file)

1895153

In [178]:

# using string methods 'startswith' and 'endswith', filter each product-active-ingredient to help map classes for each 
# medication   

for x,y in zip(present,indicies):
    if x.endswith('MAB') or x.endswith('REMICADE'):
        class_df.loc[y, 'class_id'] = 1
        class_df.loc[y, 'class'] = 'monoclonal_antibody'
        class_df.loc[y, 'indication'] = 'autoimmune diseases'
    elif x.endswith('PRIL'):
        class_df.loc[y, 'class_id'] = 2
        class_df.loc[y, 'class'] = 'ACE_inhibitor'
        class_df.loc[y, 'indication'] = 'hypertenstion' 
    elif x.endswith('TIDINE') or x.endswith('ZANTAC'):
        class_df.loc[y, 'class_id'] = 3
        class_df.loc[y, 'class'] = '2nd_gen_antihistamine'
        class_df.loc[y, 'indication'] = 'allergy'
    elif x.endswith('STATIN', 0, 12) or x.endswith('STATIN'):
        class_df.loc[y, 'class_id'] = 4
        class_df.loc[y, 'class'] = 'HMG-CoA reductase inhibitor'
        class_df.loc[y, 'indication'] = 'hyperlipidemia'
    elif x.endswith('AZEPAM') or x.endswith('ZOLAM'):
        class_df.loc[y, 'class_id'] = 5
        class_df.loc[y, 'class'] = 'benzodiazepine'
        class_df.loc[y, 'indication'] = 'anxiety'
    elif x.endswith('AFIL'):
        class_df.loc[y, 'class_id'] = 6
        class_df.loc[y, 'class'] = 'phosphodiesterase inhibitor'
        class_df.loc[y, 'indication'] = 'erectile dysfunction, hypertension'
    elif x.endswith('ANE'):
        class_df.loc[y, 'class_id'] = 7
        class_df.loc[y, 'class'] = 'inhaled anestetics'
        class_df.loc[y, 'indication'] = 'anesthesia'
    elif x.endswith('ARTAN'):
        class_df.loc[y, 'class_id'] = 8
        class_df.loc[y, 'class'] = 'angiotension receptor blocker'
        class_df.loc[y, 'indication'] = 'hypertension'
    elif x.endswith('AZINE'):
        class_df.loc[y, 'class_id'] = 9
        class_df.loc[y, 'class'] = 'phenothiazines'
        class_df.loc[y, 'indication'] = 'antipsychotic'
    elif x.endswith('AZOLE'):
        class_df.loc[y, 'class_id'] = 10
        class_df.loc[y, 'class'] = 'azole-antifungal'
        class_df.loc[y, 'indication'] = 'antifungal'
    elif x.endswith('BARBITAL'):
        class_df.loc[y, 'class_id'] = 11
        class_df.loc[y, 'class'] = 'barbituates'
        class_df.loc[y, 'indication'] = 'anxiety'
    elif x.endswith('CAINE'):
        class_df.loc[y, 'class_id'] = 12
        class_df.loc[y, 'class'] = 'local anesthetics'
        class_df.loc[y, 'indication'] = 'anesthesia'
    elif x.endswith('CILLIN'):
        class_df.loc[y, 'class_id'] = 13
        class_df.loc[y, 'class'] = 'penecillin antibiotics'
        class_df.loc[y, 'indication'] = 'antibiotic'
    elif x.endswith('CYCLINE'):
        class_df.loc[y, 'class_id'] = 14
        class_df.loc[y, 'class'] = 'tetracyclines'
        class_df.loc[y, 'indication'] = 'antibiotic'
    elif x.endswith('ETINE'):
        class_df.loc[y, 'class_id'] = 15
        class_df.loc[y, 'class'] = 'selective serotonia reuptake inhibitors'
        class_df.loc[y, 'indication'] = 'depression'
    elif x.endswith('FEB') or x.endswith('FENE'):
        class_df.loc[y, 'class_id'] = 16
        class_df.loc[y, 'class'] = 'selective estrogen response modifiers'
        class_df.loc[y, 'indication'] = 'osteoprosis, cancer treatment'
    elif x.endswith('FLOXACIN'):
        class_df.loc[y, 'class_id'] = 17
        class_df.loc[y, 'class'] = 'fluoroquinolones'
        class_df.loc[y, 'indication'] = 'antibiotics'
    elif x.endswith('FUNGIN'):
        class_df.loc[y, 'class_id'] = 18
        class_df.loc[y, 'class'] = 'echinocandins'
        class_df.loc[y, 'indication'] = 'antifungal'
    elif x.endswith('GRASTIM') or x.endswith('GRAMOSTIM'):
        class_df.loc[y, 'class_id'] = 19
        class_df.loc[y, 'class'] = 'granulocyte colony stimulating factors'
        class_df.loc[y, 'indication'] = 'blood dyscrasias'
    elif x.endswith('IDE'):
        class_df.loc[y, 'class_id'] = 20
        class_df.loc[y, 'class'] = 'loop diuretics'
        class_df.loc[y, 'indication'] = 'hypertension'
    elif x.endswith('IPINE'):
        class_df.loc[y, 'class_id'] = 21
        class_df.loc[y, 'class'] = 'dihydropyridine calcium channel blockers'
        class_df.loc[y, 'indication'] = 'hypertension'
    elif x.endswith('IPRAMINE'):
        class_df.loc[y, 'class_id'] = 22
        class_df.loc[y, 'class'] = 'tricyclic antidepressants'
        class_df.loc[y, 'indication'] = 'depression'
    elif x.endswith('IUM') or x.endswith('URONIUM'):
        class_df.loc[y, 'class_id'] = 23
        class_df.loc[y, 'class'] = 'nondepolarizing paralytics'
        class_df.loc[y, 'indication'] = 'anesthesia'
    elif x.endswith('LUKAST'):
        class_df.loc[y, 'class_id'] = 24
        class_df.loc[y, 'class'] = 'LTD receptor antagonist'
        class_df.loc[y, 'indication'] = 'asthma'
    elif x.endswith('NAVIR'):
        class_df.loc[y, 'class_id'] = 25
        class_df.loc[y, 'class'] = 'protease inhibitor'
        class_df.loc[y, 'indication'] = 'antiviral'
    elif x.endswith('OLOL'):
        class_df.loc[y, 'class_id'] = 26
        class_df.loc[y, 'class'] = 'beta blocker'
        class_df.loc[y, 'indication'] = 'hypertension'
    elif x.endswith('OXIN'):
        class_df.loc[y, 'class_id'] = 27
        class_df.loc[y, 'class'] = 'cardiac glycoside'
        class_df.loc[y, 'indication'] = 'arrhythmias'
    elif x.endswith('PYHLLINE'):
        class_df.loc[y, 'class_id'] = 28
        class_df.loc[y, 'class'] = 'methlxanthine'
        class_df.loc[y, 'indication'] = 'bronchodilator'
    elif x.endswith('QUINE'):
        class_df.loc[y, 'class_id'] = 29
        class_df.loc[y, 'class'] = 'quinolone derivatives'
        class_df.loc[y, 'indication'] = 'antimalarial'
    elif x.endswith('TECAN'):
        class_df.loc[y, 'class_id'] = 30
        class_df.loc[y, 'class'] = 'topoisomerase-1 inhibitor'
        class_df.loc[y, 'indication'] = 'chemotherapy'
    elif x.endswith('TEROL'):
        class_df.loc[y, 'class_id'] = 31
        class_df.loc[y, 'class'] = 'Beta-2 agonist'
        class_df.loc[y, 'indication'] = 'bronchodilator'
    elif x.endswith('TINE'):
        class_df.loc[y, 'class_id'] = 32
        class_df.loc[y, 'class'] = 'allylamine antifungals'
        class_df.loc[y, 'indication'] = 'antifungal'
    elif x.endswith('TOPOSIDE'):
        class_df.loc[y, 'class_id'] = 33
        class_df.loc[y, 'class'] = 'topoisomerase-2 inhibitor'
        class_df.loc[y, 'indication'] = 'chemotherapy'
    elif x.endswith('TRIPTAN'):
        class_df.loc[y, 'class_id'] = 34
        class_df.loc[y, 'class'] = '5-HT1B/1D agonist'
        class_df.loc[y, 'indication'] = 'migraines'
    elif x.endswith('TROPIN'):
        class_df.loc[y, 'class_id'] = 35
        class_df.loc[y, 'class'] = 'pituitary hormone'
        class_df.loc[y, 'indication'] = 'hormone deficiency'
    elif x.endswith('VAPTAN'):
        class_df.loc[y, 'class_id'] = 36
        class_df.loc[y, 'class'] = 'vasopressin receptor antagonist'
        class_df.loc[y, 'indication'] = 'hypertension'
    elif x.endswith('ZOSIN'):
        class_df.loc[y, 'class_id'] = 37
        class_df.loc[y, 'class'] = 'alpha-1 antagonist'
        class_df.loc[y, 'indication'] = 'hypertension, BPH'
    elif x.startswith('PREDNISONE') or x.startswith('PREDNISOLONE'):
        class_df.loc[y, 'class_id'] = 38
        class_df.loc[y, 'class'] = 'corticosteroid'
        class_df.loc[y, 'indication'] = 'immunosupressant'
    elif x.startswith('METHOTREXATE'):
        class_df.loc[y, 'class_id'] = 39
        class_df.loc[y, 'class'] = 'antimetabolites'
        class_df.loc[y, 'indication'] = 'cancer treatment'
    elif x.startswith('XARELTO') or x.startswith('WARFARIN'):
        class_df.loc[y, 'class_id'] = 40
        class_df.loc[y, 'class'] = 'anticoagulant' 
        class_df.loc[y, 'indication'] = 'blood clots'
    elif x.startswith('INFLECTRA'):
        class_df.loc[y, 'class_id'] = 41
        class_df.loc[y, 'class'] = 'TNF blocking agent'
        class_df.loc[y, 'indication'] = 'autoimmune diseases'
    elif x.startswith('ENBREL'):
        class_df.loc[y, 'class_id'] = 42
        class_df.loc[y, 'class'] = 'TNF inhibitor'
        class_df.loc[y, 'indication'] = 'autoimmune diseases'
    elif x.startswith('DEXAMETHASONE'):
        class_df.loc[y, 'class_id'] = 43
        class_df.loc[y, 'class'] = 'glucocorticoid'
        class_df.loc[y, 'indication'] = 'immunosupressant'
    elif x.startswith('AVONEX'):
        class_df.loc[y, 'class_id'] = 44
        class_df.loc[y, 'class'] = 'interferon'
        class_df.loc[y, 'indication'] = 'multiple sclerosis'
    elif x.startswith('GABAPENTIN') or x.startswith('LYRICA'):
        class_df.loc[y, 'class_id'] = 45
        class_df.loc[y, 'class'] = 'GABA analogue'
        class_df.loc[y, 'indication'] = 'anticonvulsant, fibromyalgia, nerve pain'
    elif x.startswith('AMLODIPINE'):
        class_df.loc[y, 'class_id'] = 46
        class_df.loc[y, 'class'] = 'calcium channel blocker'
        class_df.loc[y, 'indication'] = 'hypertension, chest pain'
    elif x.startswith('XELJANZ') or x.endswith('TINIB'):
        class_df.loc[y, 'class_id'] = 47
        class_df.loc[y, 'class'] = 'tyrosine kinase inhibitors'
        class_df.loc[y, 'indication'] = 'autoimmune diseases'
    elif x.endswith('ANIB'):
        class_df.loc[y, 'class_id'] = 48
        class_df.loc[y, 'class'] = 'angiogenesis inhibitor, tyrosine kinase inhibitor'
        class_df.loc[y, 'indication'] = 'cancer treatment'
    elif x.endswith('RAFENIB'):
        class_df.loc[y, 'class_id'] = 49
        class_df.loc[y, 'class'] = 'rapidly accelerated fibrosarcoma kinase inhibitor'
        class_df.loc[y, 'indication'] = 'cancer treatment'
    elif x.startswith('ORENCIA'):
        class_df.loc[y, 'class_id'] = 50
        class_df.loc[y, 'class'] = 'immunomodulator'
        class_df.loc[y, 'indication'] = 'autoimmune diseases'
    elif x.startswith('TRUVADA'):
        class_df.loc[y, 'class_id'] = 51 
        class_df.loc[y, 'class'] = 'nucleotide analog reverse transcriptase inhibitor'
        class_df.loc[y, 'indication'] = 'antiviral'
    elif x.startswith('ACETAMINOPHEN') or x.startswith('TYLENOL'):
        class_df.loc[y, 'class_id'] = 52
        class_df.loc[y, 'class'] = 'analgesic'
        class_df.loc[y, 'indication'] = 'fever reducer'
    elif x.startswith('FOLIC ACID'):
        class_df.loc[y, 'class_id'] = 53
        class_df.loc[y, 'class'] = 'vitamin'
        class_df.loc[y, 'indication'] = 'anemia, prenatal'
    elif x.startswith('OTEZLA'):
        class_df.loc[y, 'class_id'] = 54
        class_df.loc[y, 'class'] = 'phosphodiesterase'
        class_df.loc[y, 'indication'] = 'autoimmune diseases'
    elif x.startswith('ASPIRIN'):
        class_df.loc[y, 'class_id'] = 55
        class_df.loc[y, 'class'] = 'nonsteroidal anti-inflammatory drug, blood thinners'
        class_df.loc[y, 'indication'] = 'fever reducer, inflammation'
    elif x.startswith('TECFIDERA'):
        class_df.loc[y, 'class_id'] = 56
        class_df.loc[y, 'class'] = 'dimethyl fumarate, fumaric acid ester'
        class_df.loc[y, 'indication'] = 'multiple sclerosis'
    elif x.startswith('VIREAD'):
        class_df.loc[y, 'class_id'] = 57
        class_df.loc[y, 'class'] = 'nucleotide reverse transcriptase inhibitor'
        class_df.loc[y, 'indication'] = 'antiviral'
    elif x.startswith('EMTRIVA'):
        class_df.loc[y, 'class_id'] = 58
        class_df.loc[y, 'class'] = 'nucleotide reverse transcriptase inhibitor'
        class_df.loc[y, 'indication'] = 'antiviral'
    elif x.startswith('REVLIMID'):
        class_df.loc[y, 'class_id'] = 59
        class_df.loc[y, 'class'] = 'immunomodulator'
        class_df.loc[y, 'indication'] = 'cancer treatment'
    elif x.startswith('HUMIRA'):
        class_df.loc[y, 'class_id'] = 60
        class_df.loc[y, 'class'] = 'TNF blocking agent'
        class_df.loc[y, 'indication'] = 'plaque psoriasis'
    elif x.startswith('COSENTYX'):
        class_df.loc[y, 'class_id'] = 61
        class_df.loc[y, 'class'] = 'monoclonal antibody, interleukin-17A blocker'
        class_df.loc[y, 'indication'] = 'plaque psoriasis, psoratic arthritis'
    elif x.startswith('METFORMIN'):
        class_df.loc[y, 'class_id'] = 62
        class_df.loc[y, 'class'] = 'biguanides'
        class_df.loc[y, 'indication'] = 'diabetic management'
    elif x.startswith('NEULASTA'):
        class_df.loc[y, 'class_id'] = 63
        class_df.loc[y, 'class'] = 'granulocyte colony stimulating factor'
        class_df.loc[y, 'indication'] = 'febrile neutropenia'
    elif x.startswith('DUPIXENT') or x.endswith('DUPIXENT'):
        class_df.loc[y, 'class_id'] = 64
        class_df.loc[y, 'class'] = 'monoclonal antibody, IL-4 and IL-13 blocker'
        class_df.loc[y, 'indication'] = 'atopic dermatitis, add-on asthma treatment'
    elif x.startswith('XOLAIR'):
        class_df.loc[y, 'class_id'] = 65
        class_df.loc[y, 'class'] = 'monoclonal antibody, anti-IgE antibody'
        class_df.loc[y, 'indication'] = 'asthma treatment, chronic idiopathic urticaria'
    elif x.startswith('ACTEMRA'):
        class_df.loc[y, 'class_id'] = 66
        class_df.loc[y, 'class'] = 'monoclonal antibody, IL-6 receptor antagonist'
        class_df.loc[y, 'indication'] = 'rheumatoid arthritis, giant cell arteritis, cytokine release syndrome'
    elif x.startswith('OXYCONTIN'):
        class_df.loc[y, 'class_id'] = 67
        class_df.loc[y, 'class'] = 'opioid agonist'
        class_df.loc[y, 'indication'] = 'pain management'
    elif x.startswith('ELIQUIS'):
        class_df.loc[y, 'class_id'] = 68
        class_df.loc[y, 'class'] = 'factor Xa inhibitor anticoagulant'
        class_df.loc[y, 'indication'] = 'nonvalvular atrial fibrilation'
    elif x.startswith('STELARA'):
        class_df.loc[y, 'class_id'] = 69
        class_df.loc[y, 'class'] = 'monoclonal antibody, IL-12 and IL-23 antagonist'
        class_df.loc[y, 'indication'] = 'plaque psoriasis, psoriatic arthritis, chrons disease'
    elif x.startswith('SYNTHROID') or x.startswith('LEVOTHYROXINE'):
        class_df.loc[y, 'class_id'] = 70
        class_df.loc[y, 'class'] = 'hormone'
        class_df.loc[y, 'indication'] = 'hyperthyroidism, pituitary tsh suppression'
    elif x.startswith('IBUPROFEN'):
        class_df.loc[y, 'class_id'] = 71
        class_df.loc[y, 'class'] = 'analgesic, non-steriodal anti-inflammatory drug'
        class_df.loc[y, 'indication'] = 'pain management'
    elif x.startswith('TRAMADOL'):
        class_df.loc[y, 'class_id'] = 72
        class_df.loc[y, 'class'] = 'opiate analgesic/agonist'
        class_df.loc[y, 'indication'] = 'pain management'
    elif x.startswith('IBRANCE'):
        class_df.loc[y, 'class_id'] = 73
        class_df.loc[y, 'class'] = 'kinase inhibitor'
        class_df.loc[y, 'indication'] = 'cancer treatment'
    elif x.startswith('VITAMIN D3'):
        class_df.loc[y, 'class_id'] = 74
        class_df.loc[y, 'class'] = 'vitamin'
        class_df.loc[y, 'indication'] = 'osteoporosis prevention'
    elif x.startswith('METHYLPREDNISOLONE') or x.endswith('METHYLPREDNISOLONE'):
        class_df.loc[y, 'class_id'] = 75
        class_df.loc[y, 'class'] = 'glucocorticoid'
        class_df.loc[y, 'indication'] = 'endocrine disorders, rheumatic disorders'
    elif x.startswith('TRULICITY'):
        class_df.loc[y, 'class_id'] = 76
        class_df.loc[y, 'class'] = 'glp-1 receptor agonist'
        class_df.loc[y, 'indication'] = 'glycemic management'

In [170]:
class_df.class_id.value_counts()

20    77439
3     51157
1     43342
23    32972
10    28686
      ...  
48      297
16      196
58      117
36       55
75        1
Name: class_id, Length: 72, dtype: int64

In [171]:
class_df[class_df.drugname=='EMTRIVA']

,drugname,class_id,class,indication
84543,EMTRIVA,58,nucleotide reverse transcriptase inhibitor,antiviral
126871,EMTRIVA,58,nucleotide reverse transcriptase inhibitor,antiviral
159306,EMTRIVA,58,nucleotide reverse transcriptase inhibitor,antiviral
161613,EMTRIVA,58,nucleotide reverse transcriptase inhibitor,antiviral
178035,EMTRIVA,58,nucleotide reverse transcriptase inhibitor,antiviral
...,...,...,...,...
1872268,EMTRIVA,58,nucleotide reverse transcriptase inhibitor,antiviral
1872615,EMTRIVA,58,nucleotide reverse transcriptase inhibitor,antiviral
1872858,EMTRIVA,58,nucleotide reverse transcriptase inhibitor,antiviral
1874819,EMTRIVA,58,nucleotide reverse transcriptase inhibitor,antiviral


In [172]:
class_df['class'] = class_df['class'].astype(str)

ind = class_df[class_df['class'] == 'nan']

In [179]:
#TODO: some of the drugnames reappear in this file after being mapped previously. why?

ind['drugname'].value_counts()[:20]

DUPIXENT                      7779
ACTEMRA                       7191
METHYLPREDNISOLONE            4954
TACROLIMUS                    4938
PARACETAMOL                   4812
ALLOPURINOL                   4763
LATANOPROST                   4756
POMALYST                      4414
ATRIPLA                       4408
ENTRESTO                      4364
XYREM                         4244
UPTRAVI                       4242
MYCOPHENOLATE MOFETIL         4110
HYDROXYCHLOROQUINE SULFATE    3933
IMBRUVICA                     3826
SIMPONI                       3782
RANITIDINE HCI                3743
DIPHENHYDRAMINE               3713
OPSUMIT                       3622
AZITHROMYCIN ANHYDROUS        3576
Name: drugname, dtype: int64

In [ ]:
# subset DataFrame with caseid and whether the reaction stopped when drug therapy was stopped (dechal) and if the reaction
# began again after drug therapy was restarted (rechal).


subside_relapse = drug_file[['caseid', 'dechal', 'rechal']]

In [ ]:
# The majority of the observations were of class 'Unknown', followed by 'Does Not Apply'
# There were 866404 null values also recorded


subside_relapse.dechal.value_counts()

In [ ]:
subside_relapse.dechal.isnull().sum()

In [ ]:
# The majority of the observations were of class 'Unknown', followed at distance by 'Negative' and 'Does Not Apply'
# There were 1600785 null values also recorded


subside_relapse.rechal.value_counts()

In [ ]:
subside_relapse.rechal.isnull().sum()

# Examining Primary and Secondary suspect frequency

In [ ]:
# Quantify each category

new_drug_df.role_cod.value_counts()

In [ ]:
# Separate Primary and Secondary suspects  for further investigation

primary = new_drug_df[new_drug_df.role_cod == 'PS']
secondary = new_drug_df[new_drug_df.role_cod == 'SS']

In [ ]:
primary

In [ ]:
# Choose features of interest to examine trends within each Primary and Secondary Suspect drug

primary = primary[['caseid', 'drugname', 'prod_ai', 'val_vbm', 'dose_vbm', 'cum_dose_chr']]
secondary = secondary[['caseid', 'drugname', 'dose_vbm']]

In [ ]:
# reset index

primary = primary.reset_index(drop=True)
secondary = secondary.reset_index(drop=True)

In [ ]:
# Interestingly enough, ZANTAC's generic formulation is referred to as RANITIDINE and they make up the top 2 spots on the 
# Primary suspect list. They may have high interactivity with other medications, or based on a patient's medication history,
# ZANTAC (RANITIDINE) is likely to interact with their other medications

primary.drugname.value_counts()

In [ ]:
# ZANTAC is also listed as the most frequent Secondary medication of interest. One could reasonably say that taking ZANTAC
# increases the likelihood of experiencing any SAE. 

secondary.drugname.value_counts()

In [ ]:
primary[['drugname', 'dose_vbm', 'cum_dose_chr']]